In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing


# Define sampling layer

In [9]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z """

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define encoder and decoder

In [10]:
latent_dim = 3
inputlen = 3202
units = 448

# Encoder
encoder_inputs = keras.Input(shape=(inputlen,))
x = layers.Dense(units=units, activation='relu')(encoder_inputs)
x = layers.Dense(units=units//2, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(units=units//2, activation='relu')(latent_inputs)
x = layers.Dense(units=units, activation='relu')(x)
decoder_outputs = layers.Dense(inputlen, activation='linear')(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 3202)]       0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 448)          1434944     ['input_3[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 224)          100576      ['dense_5[0][0]']                
                                                                                                  
 z_mean (Dense)                 (None, 3)            675         ['dense_6[0][0]']                
                                                                                            

# Define VAE

In [11]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #print("reconstruction = " + str(reconstruction))
            #print("z_mean = " + str(z_mean)+"\n")
            #print("z_log_var = " + str(z_log_var)+"\n")
            #print("z = " + str(z)+"\n")
            
            reconstruction_loss = keras.losses.mean_squared_error(data, reconstruction)
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            #print("First kl_loss = "+str(kl_loss)+"\n")
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss,axis=1)) 
            #mprint("Second kl_loss = "+str(kl_loss)+"\n")
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# Load dataset

In [12]:
df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/MotifsDataAutoencoder/Data/3_46cols_EnvFeatures173samples_plus_3202EC_V2_original.csv",sep=",")
df = df.iloc[:,46-len(df.columns):]
print(df.shape)



(173, 3202)


# Train VAE

In [13]:


tb = keras.callbacks.TensorBoard(log_dir="./vae_tb",update_freq="epoch")
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=500)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(df, epochs=5000, batch_size=128, callbacks=[tb,earlystop] )
vae.save_weights('./vae_checkpoint/my_checkpoint')

Epoch 1/5000
2/2 [==============================] - 0s 8ms/step - loss: 0.0723 - reconstruction_loss: 0.0097 - kl_loss: 0.1322
Epoch 2/5000
2/2 [==============================] - 0s 8ms/step - loss: 0.0389 - reconstruction_loss: 0.0078 - kl_loss: 0.0278
Epoch 3/5000
2/2 [==============================] - 0s 8ms/step - loss: 0.0401 - reconstruction_loss: 0.0065 - kl_loss: 0.0275
Epoch 4/5000
2/2 [==============================] - 0s 9ms/step - loss: 0.0264 - reconstruction_loss: 0.0056 - kl_loss: 0.0184
Epoch 5/5000
2/2 [==============================] - 0s 8ms/step - loss: 0.0156 - reconstruction_loss: 0.0047 - kl_loss: 0.0088
Epoch 6/5000
2/2 [==============================] - 0s 8ms/step - loss: 0.0095 - reconstruction_loss: 0.0044 - kl_loss: 0.0049
Epoch 7/5000
2/2 [==============================] - 0s 9ms/step - loss: 0.0100 - reconstruction_loss: 0.0041 - kl_loss: 0.0057
Epoch 8/5000
2/2 [==============================] - 0s 9ms/step - loss: 0.0085 - reconstruction_loss: 0.0033 - 